# EDA for a randomly selected user

## Load packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as metrics

## Load random user data

In [ ]:
user_number = 5
path = f"data/S{user_number:02d}_data.parquet" # input path to a parquet

df = pd.read_parquet(path)

In [ ]:
df.columns

In [ ]:
df.head()

## Pupil diameter data

---

In [ ]:
pupil_cols = [col for col in df.columns if col.startswith("pupil_diameter")]
cols_to_keep = ["pupil_timestamp", "eye_id"] + pupil_cols
df_pupil = df[cols_to_keep]

In [ ]:
df_pupil.head()

In [ ]:
df[pupil_cols].isna().sum()

Considering how few NaN values there are in the pupil diameter data, we can simply drop them.

In [ ]:
df_pupil = df_pupil.dropna()
df_pupil = df_pupil.reset_index(drop=True)
df_pupil.head(n=10)

In [ ]:
df.shape, df_pupil.shape

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

eye0_data = df_pupil[df_pupil["eye_id"] == 0]
sns.lineplot(data=eye0_data, x="pupil_timestamp", y="pupil_diameter_mean", color="blue", alpha=0.7, ax=axs[0])
axs[0].set_title("Pupil Diameter Mean Over Time - Eye ID 0")
axs[0].set_ylabel("Pupil Diameter Mean")

eye1_data = df_pupil[df_pupil["eye_id"] == 1]
sns.lineplot(data=eye1_data, x="pupil_timestamp", y="pupil_diameter_mean", color="red", alpha=0.7, ax=axs[1])
axs[1].set_title("Pupil Diameter Mean Over Time - Eye ID 1")
axs[1].set_xlabel("Timestamp")
axs[1].set_ylabel("Pupil Diameter Mean")

plt.tight_layout()
plt.show()

In [ ]:
eye1 = df_pupil["eye_id"] == 0
eye2 = df_pupil["eye_id"] == 1
df_pupil.loc[eye1]["pupil_diameter_mean"].shape, df_pupil.loc[eye2]["pupil_diameter_mean"].shape

There is a very notable difference in the number of samples between the two conditions. We will try to fix this unbalance by rounding the timestep and selecting the data for the pupils from which the both eyes have a value at the same time.

In [ ]:
df_pupil_2 = df_pupil.copy()

In [ ]:
df_pupil_2["pupil_timestamp"] = df_pupil_2["pupil_timestamp"].dt.round("10ms")
df_pupil_2 = df_pupil_2.drop_duplicates(subset=["pupil_timestamp", "eye_id"])

In [ ]:
counts = df_pupil_2.groupby("pupil_timestamp")["eye_id"].nunique()
valid_timestamps = counts[counts == 2].index
df_pupil = df_pupil_2[df_pupil_2["pupil_timestamp"].isin(valid_timestamps)]

In [ ]:
eye1 = df_pupil["eye_id"] == 0
eye2 = df_pupil["eye_id"] == 1
df_pupil.loc[eye1]["pupil_diameter_mean"].shape, df_pupil.loc[eye2]["pupil_diameter_mean"].shape

In [ ]:
metrics.root_mean_squared_error(
    df_pupil.loc[eye1]["pupil_diameter_mean"],
    df_pupil.loc[eye2]["pupil_diameter_mean"]
)

The RMSE of the pupil diameter is very low, so we can assume that the data is precise.

In [ ]:
balanced_df = pd.concat([df_pupil.loc[eye1], df_pupil.loc[eye2]], axis=0, ignore_index=True).sort_values(
    by=["pupil_timestamp"], ignore_index=True)

In [ ]:
balanced_df.describe()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12,12))
plt.suptitle("Exploratory Plots for Balanced Pupil Data", fontsize=18)

# 1. Histogram: pupil_diameter_mean distribution by eye_id
sns.histplot(
    data=balanced_df,
    x="pupil_diameter_mean",
    hue="eye_id",
    kde=True,
    ax=axs[0, 0],
    bins=50,
    element="step"
)
axs[0, 0].set_title("Distribution of Pupil Diameter Mean")
axs[0, 0].set_xlabel("Pupil Diameter Mean")

# 2. Boxplot: pupil_diameter_mean by eye_id
sns.boxplot(
    data=balanced_df,
    x="eye_id",
    y="pupil_diameter_mean",
    ax=axs[0, 1]
)
axs[0, 1].set_title("Pupil Diameter Mean by Eye")
axs[0, 1].set_xlabel("Eye ID")
axs[0, 1].set_ylabel("Pupil Diameter Mean")

# 3. Scatterplot: mean vs variance, colored by eye_id
sns.scatterplot(
    data=balanced_df.sample(5000, random_state=0),  # sample for speed
    x="pupil_diameter_mean",
    y="pupil_diameter_variance",
    hue="eye_id",
    alpha=0.5,
    ax=axs[1, 0]
)
axs[1,0].set_title("Mean vs Variance of Pupil Diameter")
axs[1,0].set_xlabel("Pupil Diameter Mean")
axs[1,0].set_ylabel("Pupil Diameter Variance")

# 4. Correlation heatmap
corr = balanced_df[
    ["pupil_diameter_mean", "pupil_diameter_variance", "pupil_diameter_skewness", "pupil_diameter_kurtosis"]
].corr()
sns.heatmap(
    corr,
    annot=True,
    cmap="coolwarm",
    ax=axs[1,1]
)
axs[1,1].set_title("Correlation Heatmap")

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

In [ ]:
sns.pairplot(
    balanced_df.sample(2000, random_state=1),
    vars=[
        "pupil_diameter_mean",
        "pupil_diameter_variance",
        "pupil_diameter_skewness",
        "pupil_diameter_kurtosis"
    ],
    hue="eye_id",
    corner=True,
    plot_kws={"alpha": 0.5, "s": 20}
)

Based on the correlation heatmap, we can see that using both kurtosis and skewness is redundant. Hence we will only use the kurtosis of the pupil diameter data, since skewness is much more correlated to variance than kurtosis.

In [ ]:
df_cleaned_1= balanced_df.copy()
df_cleaned_1 = df_cleaned_1.drop(columns=["pupil_diameter_skewness"])
df_cleaned_1.head()

In [ ]:
eye1 = df_cleaned_1["eye_id"] == 0
eye2 = df_cleaned_1["eye_id"] == 1

In [ ]:
diameter_variance_error = metrics.root_mean_squared_error(
    df_cleaned_1.loc[eye1]["pupil_diameter_variance"],
    df_cleaned_1.loc[eye2]["pupil_diameter_variance"]
)
diameter_kurtosis_error = metrics.root_mean_squared_error(
    df_cleaned_1.loc[eye1]["pupil_diameter_kurtosis"],
    df_cleaned_1.loc[eye2]["pupil_diameter_kurtosis"]
)
diameter_mean_error = metrics.root_mean_squared_error(
    df_cleaned_1.loc[eye1]["pupil_diameter_mean"],
    df_cleaned_1.loc[eye2]["pupil_diameter_mean"]
)
diameter_variance_error, diameter_kurtosis_error, diameter_mean_error

Since the RMSE between the pupils is very low, we can use the mean of the two pupils as a single value for the pupil diameter to further reduce the number of redundant features.

In [ ]:
pivoted = df_cleaned_1.pivot(index="pupil_timestamp", columns="eye_id", values=["pupil_diameter_variance", "pupil_diameter_kurtosis", "pupil_diameter_mean"])

pupils_diameter_mean = pivoted["pupil_diameter_mean"].mean(axis=1)

df_pupils_combined = pd.DataFrame({
    "pupil_timestamp": pivoted.index,
    "pupils_diameter_mean": pupils_diameter_mean,
    "pupil_diameter_variance_0": pivoted["pupil_diameter_variance"][0],
    "pupil_diameter_kurtosis_0": pivoted["pupil_diameter_kurtosis"][0],
    "pupil_diameter_variance_1": pivoted["pupil_diameter_variance"][1],
    "pupil_diameter_kurtosis_1": pivoted["pupil_diameter_kurtosis"][1],
}).reset_index(drop=True)

df_pupils_combined.head()

In [ ]:
df_pupils_combined.shape

In [ ]:
corr = df_pupils_combined[
    ["pupils_diameter_mean", "pupil_diameter_variance_0","pupil_diameter_variance_1", "pupil_diameter_kurtosis_0", "pupil_diameter_kurtosis_1"]
].corr()

sns.heatmap(
    corr,
    annot=True,
    cmap="coolwarm",
)

From this correlation heatmap we can see that we don't need two values for kurtosis and variance for each pupil, since they have correlation = 1.0. We can drop one of them.

In [ ]:
df_fin = df_pupils_combined.copy()
df_fin = df_fin.drop(columns=["pupil_diameter_variance_1", "pupil_diameter_kurtosis_1"])
df_fin.head()

In [ ]:
corr_fin = df_fin[
    ["pupils_diameter_mean", "pupil_diameter_variance_0", "pupil_diameter_kurtosis_0"]
].corr()

sns.heatmap(
    corr_fin,
    annot=True,
    cmap="coolwarm",
)

In [ ]:
import itertools

cols = ["pupils_diameter_mean", "pupil_diameter_variance_0", "pupil_diameter_kurtosis_0"]
pairs = list(itertools.combinations(cols, 2))

fig, axs = plt.subplots(1, len(pairs), figsize=(6 * len(pairs), 5))

for i, (x, y) in enumerate(pairs):
    sns.scatterplot(data=df_fin, x=x, y=y, alpha=0.5, ax=axs[i], 
                   color='hotpink', marker='x', s=50)
    axs[i].set_title(f"{x} vs {y}")

plt.tight_layout()
plt.show()

---

## Drop rows where valence or arousal is nan

In [ ]:
# Drop rows where valence or arousal or trigger are NaN

df = df.dropna(subset=['valence_rating', 'arousal_rating', 'trigger'])

In [ ]:
# 1) make sure ratings are numeric
df['valence_rating']  = pd.to_numeric(df['valence_rating'],  errors='coerce')
df['arousal_rating']  = pd.to_numeric(df['arousal_rating'], errors='coerce')

# 2) drop any rows that became NaN after coercion
df = df.dropna(subset=['valence_rating', 'arousal_rating', 'trigger'])

In [ ]:
# valence_per_trigger = df.groupby('trigger').mean()["valence_rating"].to_frame().reset_index()

# arousal_per_trigger = df.groupby('trigger').mean()["arousal_rating"].to_frame().reset_index()

## Data filtering
### Filter data according to recommended criteria

In [ ]:
original_len = len(df)

keep_mask = pd.Series(True, index=df.index)

# 1) For pupil diameter: Remove blinks (no pupil size data)
keep_mask &= df["diameter"].notna()

keep_mask.sum() / len(keep_mask)

In [ ]:
# 2) For fixations: Filter out eye positions outside stimulus/field of view
valid_positions = (
    (df["norm_pos_x_fixation"] >= -2)
    & (df["norm_pos_x_fixation"] <= 2)
    & (df["norm_pos_y_fixation"] >= -2)
    & (df["norm_pos_y_fixation"] <= 2)
)
keep_mask &= (
    valid_positions | df["norm_pos_x_fixation"].isna()
)

keep_mask.sum() / len(keep_mask)

In [ ]:
df["HR"].min(), df["HR"].max(), df["HR"].isna().sum()

In [ ]:
# 3) For HR: Filter out abnormal values
valid_hr = (df["HR"] <= 150) & (df["HR"] >= 50) | df["HR"].isna()
keep_mask &= valid_hr

keep_mask.sum() / len(keep_mask)

In [ ]:
# Calculate HR jumps
hr_jumps = df["HR"].diff().abs()
keep_mask &= (hr_jumps <= 20) | hr_jumps.isna()

keep_mask.sum() / len(keep_mask)

In [ ]:
# 4) For EDA: Remove zero signal
keep_mask &= (df["EDA"] != 0) | df["EDA"].isna()

keep_mask.sum() / len(keep_mask)

In [ ]:
# Apply all filters
df_filtered = df[keep_mask]

In [ ]:
print(f"Original dataset: {original_len} rows")
print(f"Filtered dataset: {len(df_filtered)} rows")
print(
    f"Removed: {original_len - len(df_filtered)} rows ({(1 - len(df_filtered) / original_len) * 100:.2f}%)"
)

In [ ]:
df = df_filtered

### Valence and Arousal rating distribution

In [ ]:
# now compute means per trigger on just that one column
valence_per_trigger = df.groupby('trigger')['valence_rating'].mean().reset_index()
arousal_per_trigger = df.groupby('trigger')['arousal_rating'].mean().reset_index()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

sns.histplot(valence_per_trigger["valence_rating"], ax=axs[0])
axs[0].set_title("Valence rating distribution")
axs[0].set_xlabel("Valence rating")

sns.histplot(arousal_per_trigger["arousal_rating"], ax=axs[1])
axs[1].set_title("Arousal rating distribution")
axs[1].set_xlabel("Arousal rating")

plt.show()

### Bitalino in time

In [ ]:
df_sorted = df.sort_values(by=['pupil_timestamp'])

# Get row every 100 rows in pupil_timestamp
df_sorted = df_sorted.iloc[::100, :]

# Plot EDA in time
fig, axs = plt.subplots(2, 2, figsize=(15, 10))
plt.suptitle("Bitalino data in time")

sns.lineplot(data=df_sorted, x="pupil_timestamp", y="EDA", ax=axs[0, 0])
axs[0, 0].set_title("EDA in time")
axs[0, 0].set_xlabel("Time (s)")
axs[0, 0].set_ylabel("EDA")

sns.lineplot(data=df_sorted, x="pupil_timestamp", y="HR", ax=axs[0, 1])
axs[0, 1].set_title("HR in time")
axs[0, 1].set_xlabel("Time (s)")
axs[0, 1].set_ylabel("HR")

sns.lineplot(data=df_sorted, x="pupil_timestamp", y="EKG", ax=axs[1, 0])
axs[1, 0].set_title("EKG in time")
axs[1, 0].set_xlabel("Time (s)")
axs[1, 0].set_ylabel("EKG")

sns.lineplot(data=df_sorted, x="pupil_timestamp", y="light", ax=axs[1, 1])
axs[1, 1].set_title("Light in time")
axs[1, 1].set_xlabel("Time (s)")
axs[1, 1].set_ylabel("Light")


plt.show()

In [ ]:
# X, Y of pupil scatter plot

fig, axs = plt.subplots(1, 2, figsize=(15, 5))

df_fixated = df[df["norm_pos_y_fixation"] > 0]
sns.scatterplot(data=df_fixated[::100], x="norm_pos_x_fixation", y="norm_pos_y_fixation", ax=axs[0])
axs[0].set_title("Pupil X, Y scatter plot")
axs[0].set_xlabel("Pupil X")
axs[0].set_ylabel("Pupil Y")

sns.scatterplot(data=df_fixated[::100], x="norm_pos_x_fixation", y="norm_pos_y_fixation", hue="trigger", ax=axs[1])
axs[1].set_title("Pupil X, Y scatter plot with trigger")
axs[1].set_xlabel("Pupil X")
axs[1].set_ylabel("Pupil Y")    

plt.show()

In [ ]:
df

In [ ]:
# Print correlation matrix

columns_of_interest = ["pupil_timestamp", "eye_id", "confidence", "norm_pos_x", "norm_pos_y", "diameter", "EKG", "light", "EDA", "HR", "trigger", "valence_rating", "arousal_rating"]
corr = df[columns_of_interest].corr(numeric_only=True)

fig, ax = plt.subplots(figsize=(15, 10))
sns.heatmap(corr, annot=True, ax=ax, cmap="coolwarm")
plt.show()

In [ ]:
# Confidence level distribution

fig, axs = plt.subplots(1, 2, figsize=(15, 5))

sns.histplot(df["confidence"], ax=axs[0])
axs[0].set_title("Confidence level distribution")
axs[0].set_xlabel("Confidence level")

sns.histplot(df["confidence"], ax=axs[1], cumulative=True)
axs[1].set_title("Confidence level distribution (cumulative)")
axs[1].set_xlabel("Confidence level")

plt.show()

In [ ]:
# Pupil size distribution

fig, axs = plt.subplots(1, 2, figsize=(15, 5))

sns.histplot(df["diameter"], ax=axs[0])
axs[0].set_title("Pupil size distribution")
axs[0].set_xlabel("Pupil size")

sns.histplot(df["diameter"], ax=axs[1], cumulative=True)
axs[1].set_title("Pupil size distribution (cumulative)")
axs[1].set_xlabel("Pupil size")

plt.show()

## Scatter plots of Arousal vs Bitalino data

In [ ]:
# Arousal vs EDA, EKG, HR, light

fig, axs = plt.subplots(4, 2, figsize=(15, 20))

sns.scatterplot(data=df, x="EDA", y="arousal_rating", ax=axs[0, 0])
axs[0, 0].set_title("EDA vs Arousal")
axs[0, 0].set_xlabel("EDA")
axs[0, 0].set_ylabel("Arousal")

sns.scatterplot(data=df, x="EDA", y="valence_rating", ax=axs[0, 1])
axs[0, 1].set_title("EDA vs Valence")
axs[0, 1].set_xlabel("EDA")
axs[0, 1].set_ylabel("Arousal")

sns.scatterplot(data=df, x="EKG", y="arousal_rating", ax=axs[1, 0])
axs[1, 0].set_title("EKG vs Arousal")
axs[1, 0].set_xlabel("EKG")
axs[1, 0].set_ylabel("Arousal")

sns.scatterplot(data=df, x="EKG", y="valence_rating", ax=axs[1, 1])
axs[1, 1].set_title("EKG vs Valence")
axs[1, 1].set_xlabel("EKG")
axs[1, 1].set_ylabel("Valence")

sns.scatterplot(data=df, x="HR", y="arousal_rating", ax=axs[2, 0])
axs[2, 0].set_title("HR vs Arousal")
axs[2, 0].set_xlabel("HR")
axs[2, 0].set_ylabel("Arousal")

sns.scatterplot(data=df, x="HR", y="valence_rating", ax=axs[2, 1])
axs[2, 1].set_title("HR vs Valence")
axs[2, 1].set_xlabel("HR")
axs[2, 1].set_ylabel("Valence")

sns.scatterplot(data=df, x="light", y="arousal_rating", ax=axs[3, 0])
axs[3, 0].set_title("Light vs Arousal")
axs[3, 0].set_xlabel("Light")
axs[3, 0].set_ylabel("Arousal")

sns.scatterplot(data=df, x="light", y="valence_rating", ax=axs[3, 1])
axs[3, 1].set_title("Light vs Valence")
axs[3, 1].set_xlabel("Light")
axs[3, 1].set_ylabel("Valence")

plt.show()


In [ ]:
df.shape